In [22]:
import numpy as np
import pandas as pd
from copy import copy
import openpyxl
import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')

In [33]:
data = pd.read_csv('DATA_STUDENTS.csv', header=None)

new_header = data.iloc[458]

data = data.rename(columns=new_header)

data = data.drop(459).reset_index(drop=True)

print(f"\nТипы данных в столбцах: \n{data.dtypes}")

#Данные, исходя из вывода имеют тип object. Меняем его на нормальный.
numeric_cols = ['Age_Y', 'Weight_Kg', 'Height_cm', 'BMI', 'Speed_m/s', 'Jump _distance_cm']
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

print(f"\nТипы данных в столбцах: \n{data.dtypes}")



Типы данных в столбцах: 
ID                   object
Age_Y                object
Gender               object
Weight_Kg            object
Height_cm            object
BMI                  object
Speed_m/s            object
Jump _distance_cm    object
Region               object
dtype: object

Типы данных в столбцах: 
ID                    object
Age_Y                float64
Gender                object
Weight_Kg            float64
Height_cm            float64
BMI                  float64
Speed_m/s            float64
Jump _distance_cm    float64
Region                object
dtype: object


In [24]:
data

,ID,Age_Y,Gender,Weight_Kg,Height_cm,BMI,Speed_m/s,Jump _distance_cm,Region
0,371,12,male,30,138,16,6.17,321,Bogota
1,636,13,male,31,141,16,4.81,280,Bogota
2,666,13,male,31,142,15,5.43,310,Bogota
3,1548,13,male,31,142,15,4.7,320,Bogota
4,358,13,male,31,145,15,5.62,245,Bogota
...,...,...,...,...,...,...,...,...,...
1852,106,12,male,42.5,160,17,5.41,360,Bogota
1853,1693,11,male,42.5,160,17,5.4,360,Cairo
1854,1611,15,female,50.3,162,19,5.4,243,Riga
1855,310,14,female,58.5,155,24,5.33,270,Riga


In [35]:
# Размерность данных
print(f"Размер данных: {data.shape}")

# Пропуски
print("\nКоличество пропущенных значений:")
print(data.isnull().sum())

Размер данных: (1857, 9)

Количество пропущенных значений:
ID                   0
Age_Y                1
Gender               0
Weight_Kg            1
Height_cm            1
BMI                  1
Speed_m/s            8
Jump _distance_cm    1
Region               0
dtype: int64


Удаляем пропуски


In [26]:
# Удаляем строки с пропущенными значениями
data = data.dropna()

# Сбрасываем индексы после удаления строк
data = data.reset_index(drop=True)

# Проверяем количество пропусков после очистки
print("\nКоличество пропущенных значений после очистки:")
print(data.isnull().sum())


Количество пропущенных значений после очистки:
ID                   0
Age_Y                0
Gender               0
Weight_Kg            0
Height_cm            0
BMI                  0
Speed_m/s            0
Jump _distance_cm    0
Region               0
dtype: int64


Межквартильный размах определяется через квантили - разница между 75-м и 25-м процентилям.

In [ ]:
#Да, функция для одного применения, но я предпочту сделать так, потому что можно сунуть в библиотеку.
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df.reset_index(drop=True)

# фильтрация
data_clean = remove_outliers_iqr(data, numeric_cols)

# Проверка размерности
print("До удаления выбросов:", data.shape)
print("После удаления выбросов:", data_clean.shape)

До удаления выбросов: (1850, 9)
После удаления выбросов: (1689, 9)
